In [1]:
import os, sys
cur_dir = os.getcwd()
basename = os.path.basename(cur_dir)
for _ in range(5):
    if basename != 'OCLog':
        cur_dir = os.path.dirname(cur_dir)
        basename = os.path.basename(cur_dir)
        #print(cur_dir, basename)
    else:
        if cur_dir not in sys.path:
            sys.path.append(cur_dir)
            #print(sys.path)
import os
from openpyxl import Workbook
from openpyxl import load_workbook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tqdm import trange, tqdm, tnrange
from oclog.BGL.bglv1 import BGLog
from oclog.openset.opensetv12 import OpenSet
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
import sklearn.metrics as m
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.manifold import TSNE
tf.random.set_seed(1234)
np.random.seed(1234) 

In [2]:
oset = OpenSet()
train_data, val_data,  test_data, bglog  = oset.get_bgdata(bg_class_obj=BGLog, save_padded_num_sequences=False, 
                                                           load_from_pkl=True, ablation=1500,
                                                           debug=False,
                                                          designated_ukc_cls=5)

padded_num_seq_df loaded from C:\Users\Bhujay_ROG\MyDev\OCLog\oclog\BGL\data\bgl_ukc.pkl
trained tokenizer, tk, loaded from C:\Users\Bhujay_ROG\MyDev\OCLog\oclog\BGL\data\bgl_tk.pkl
train_0:, 1200, val_0:, 150, test_0:, 150, train_1:, 1200, val_1:, 150, test_1:, 150, train_2:, 1200, val_2:, 150, test_2:, 150, class 5 is added as ukc
ukc_5:, 150
length of train_data - (num_seq_per_cls * num_class)// batch size: 112
get_bgdata  num_classses: 3 and self.num_classes: 3


In [3]:
# res = oset.train(train_data=train_data, val_data=val_data,
#                  test_data=test_data, bglog=bglog, 
#                  manual_color_map=True, centroid_black=True,                               
#                  tsne_n_iter=2000, tsne_perplexity=8,
#                  embedding_size=12, oc_lr=2.5, pt_epochs=2, oc_epochs=2,
#                 save_ptmodel=False, save_ocmodel=False, oc_centroid_plot=False)

In [4]:
from oclog.openset.ptmodelv1 import LogLineEncoder, LogSeqEncoder, LogClassifier

In [5]:
chars_in_line = train_data.element_spec[0].shape[2]
line_in_seq = train_data.element_spec[0].shape[1]
num_classes = train_data.element_spec[1].shape[1]
char_embedding_size = len(bglog.tk.word_index) 
pt_optimizer = 'adam'
pt_loss = 'categorical_crossentropy'
pt_metrics = ['accuracy', tf.keras.metrics.Precision(),tf.keras.metrics.Recall()]
tf_random_seed =  1234 
embedding_size = 16
# num_classes = kwargs.get('num_classes', train_data.element_spec[1].shape[1])
tf.random.set_seed(tf_random_seed)
line_encoder = LogLineEncoder(bglog, chars_in_line=chars_in_line, char_embedding_size=char_embedding_size,)
log_seqencer =  LogSeqEncoder(line_in_seq=line_in_seq, dense_neurons=embedding_size)
ptmodel_arch = LogClassifier(line_encoder=line_encoder, seq_encoder=log_seqencer, num_classes=num_classes)
# ptmodel_arch.compile(optimizer=pt_optimizer, loss=pt_loss, metrics=pt_metrics)
ptmodel_arch.compile(optimizer=pt_optimizer, metrics=pt_metrics)
ptmodel_arch.fit(train_data)
# ptmodel_arch.batch_features

vocab_size: 50
in train step
label_batch withi train step:...... Tensor("IteratorGetNext:1", shape=(32, 3), dtype=float32)
in train step
label_batch withi train step:...... Tensor("IteratorGetNext:1", shape=(32, 3), dtype=float32)
112/112 [==============================] - 12s 100ms/step - accuracy: 0.9342 - precision_1: 0.9903 - recall_1: 0.8786


In [6]:
ptmodel_arch.batch_features
batch = next(iter(train_data))
logseq_batch, label_batch = batch

In [7]:
print(logseq_batch.shape, label_batch.shape)
ptmodel_arch(logseq_batch)
batch_features = ptmodel_arch.batch_features
print('batch_features', batch_features.shape)
print('first feature', batch_features[0])
print('')
centroids = tf.zeros((num_classes, embedding_size))
total_labels = tf.zeros(num_classes)
for i in range(label_batch.shape[0]): # (32, 4) --> here length is 32
    label = label_batch[i] # label looks like [0 0 0 1]
    # numeric_label = np.argmax(label) # index position of the label = 3 , so it is actually class =3
    numeric_label = tf.math.argmax(label).numpy()
    ##total_labels = [0 0 0 0] each col representing a class 
    ## count the number for each class
    total_labels_lst = tf.unstack(total_labels)
    total_labels_lst[numeric_label] += 1 
    total_labels = tf.stack(total_labels_lst)
    centroids_lst = tf.unstack(centroids)
    centroids_lst[numeric_label] += batch_features[i]
    centroids = tf.stack(centroids_lst)
    # self.labelled_features[numeric_label] = features[i]
    # each row index in the centroid array is a class
    # we add first identify the feature belonging to which class by the numeric_label
    # Then add all the features belonging to the class in the corresponding row of the centroid arr
### shape of centroids is (4, 16) whereas shape of total_labels is (1, 4)
### reshape the total_labels as 4,1 ==> [[0], [0], [0], [0]]==> 4 rows 
## so that we can divide the centroids array by the total_labels
total_label_reshaped = tf.reshape(total_labels, (num_classes, 1))
centroids /= total_label_reshaped
pt_batch_centroids = centroids

(32, 32, 64) (32, 3)
batch_features (32, 16)
first feature tf.Tensor(
[ 0.         0.         0.        10.295653   4.5766234  0.
  0.         0.2313345  0.         0.         0.         0.
 13.923516   0.         0.         0.       ], shape=(16,), dtype=float32)



In [8]:
numeric_label.numpy()
# type(numeric_label.numpy())

AttributeError: 'numpy.int64' object has no attribute 'numpy'

In [9]:
print('centroid for 3 classes', pt_batch_centroids)
# print('centroid for 32 features', pt_batch_centroids[0])
label_indexs = tf.math.argmax(label_batch, axis=1)
# c = tf.gather(centroids, indices=label_indexs)
centroid_for_features_as_per_class = tf.gather(centroids, indices=label_indexs)
print('centroid_for_features_as_per_class', centroid_for_features_as_per_class.shape)
print('first feature in batch_feature', batch_features[0], )
euc_dis = tf.norm(batch_features - centroid_for_features_as_per_class, ord='euclidean', axis=1)
print('distance for the first feature', euc_dis[0], euc_dis.shape)

centroid for 3 classes tf.Tensor(
[[0.0000000e+00 0.0000000e+00 0.0000000e+00 1.0461287e+01 4.5569844e+00
  0.0000000e+00 0.0000000e+00 1.9337600e-01 5.5942935e-04 3.8482281e-03
  0.0000000e+00 0.0000000e+00 1.4086994e+01 0.0000000e+00 0.0000000e+00
  0.0000000e+00]
 [1.9430670e+01 2.0913212e+01 4.3582392e+00 1.8160439e+00 2.1616616e+00
  1.4756054e-01 0.0000000e+00 0.0000000e+00 3.1542070e+00 5.4978263e-01
  1.6582438e-01 3.3761695e-01 4.6369295e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00]
 [3.3319256e-01 4.7300778e-02 1.4957752e+01 4.8455632e-01 9.1551437e+00
  0.0000000e+00 0.0000000e+00 5.6322670e-01 1.3443686e+01 1.4567280e+01
  5.8653355e+00 0.0000000e+00 7.5367551e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00]], shape=(3, 16), dtype=float32)
centroid_for_features_as_per_class (32, 16)
first feature in batch_feature tf.Tensor(
[ 0.         0.         0.        10.295653   4.5766234  0.
  0.         0.2313345  0.         0.         0.         0.
 13.923516   0.         0.   

In [10]:
tf.reduce_mean(euc_dis, )

<tf.Tensor: shape=(), dtype=float32, numpy=7.984352>